In [ ]:
# Ensure the svt library will auto reload (useful for development work)
%load_ext autoreload

In [ ]:
# Imports
import sys, math, os.path
import numpy as np
import svt
%autoreload

# Seed random number generator for consistency
np.random.seed(0)
asset_dir = os.path.join("..", "..", "ci", "assets")

In [ ]:
# Hello world example

# Create a Scene
scene = svt.Scene()

# Load some images
def create_image_mesh(filename, layer_id):
    image = scene.create_image()
    image.load(os.path.join(asset_dir, filename))
    mesh = scene.create_mesh(texture_id = image.image_id, layer_id = layer_id, nn_texture = False)
    mesh.add_image(x_axis = np.array([[2.0, 0.0, 0.0]]))
    return mesh

mesh_hello = create_image_mesh("Hello.png", "Hello")
mesh_analog = create_image_mesh("Analog.png", "Analog Science")
mesh_science = create_image_mesh("Science.png", "Analog Science") # Note re-use of layer_id to group with mesh_analog

# Load audio track
audio_hello = scene.create_audio()
audio_hello.load(os.path.join(asset_dir, "hello.mp3"))

# Load hand mesh vertices and triangles from an OBJ
hand_mesh = svt.load_obj(os.path.join(asset_dir, "hand.obj"))

# Loop subdivide this hand mesh
cmvs, cmts = svt.LoopSubdivStencil(hand_mesh.triangles, 1, True).apply(hand_mesh.positions)

# Create an ScenePic hand mesh object with and without subdivision
model_mesh_lo = scene.create_mesh(shared_color = svt.Color(1.0, 0.0, 0.0))
model_mesh_lo.add_mesh(hand_mesh, reverse_triangle_order = True) # Normals computed automatically
model_mesh_hi = scene.create_mesh(shared_color = svt.Color(0.0, 0.0, 1.0))
model_mesh_hi.add_mesh_without_normals(cmvs, cmts, reverse_triangle_order = True) # Normals computed automatically

# Create canvases
Size = 500
canvas_hello = scene.create_canvas_3d(width=Size, height=Size, audio_id=audio_hello.audio_id) # Uses default camera and shading parameters
hand_center = np.mean(hand_mesh.positions, axis = 0)
hand_camera = svt.Camera(center = hand_center + np.array([0.0, 0.0, 0.5]), look_at = hand_center, up_dir = np.array([0.0, 1.0, 0.0]), fov_y_degrees = 45.0)
hand_shading = svt.Shading(bg_color=svt.Colors.White)
hand_ui_parameters = svt.UIParameters()
canvas_hand = scene.create_canvas_3d(width = Size, height = Size, camera = hand_camera, shading = hand_shading, ui_parameters = hand_ui_parameters)

# Create some frames in an animation
n_frames = 30
for i in range(n_frames):
    angle = 2 * math.pi * i / n_frames
    
    # Add some meshes with transforms
    frame_hello = canvas_hello.create_frame()
    frame_hello.add_mesh(mesh_hello, transform = svt.Transforms.Translate([math.cos(angle), math.sin(angle), 0]))
    frame_hello.add_mesh(mesh_analog, transform = svt.Transforms.Translate([math.cos(angle - 0.5 * math.pi), math.sin(angle - 0.5 * math.pi), -0.5]))
    frame_hello.add_mesh(mesh_science, transform = svt.Transforms.Translate([math.cos(angle + math.pi), math.sin(angle + math.pi), -1.0]))

    # Add some further primitives randomized per frame
    mesh_primitives = scene.create_mesh(layer_id = "Primitives")
    mesh_primitives.add_sphere(color = svt.Color(0,1,0), transform = svt.Transforms.Scale(0.2 + 0.2 * (1 + math.cos(angle))), fill_triangles = False, add_wireframe = True)
    mesh_primitives.add_cube(color = svt.Color(0,0,1), transform = svt.Transforms.Translate([-1.0, -1.0, -3.0]))
    frame_hello.add_mesh(mesh_primitives)
    
    # Add a noise pointcloud
    mesh_noise = scene.create_mesh(shared_color = svt.Color(1.0, 0.0, 0.0), layer_id = "Noise")
    mesh_noise.add_cylinder() # The primitive for the pointcloud
    mesh_noise.apply_transform(svt.Transforms.Scale([0.02, 0.1, 0.1])) # Squash this to the right size
    mesh_noise.apply_transform(svt.Transforms.RotationToRotateXAxisToAlignWithAxis(np.array([0.5, 0.5, 0.5]))) # Rotate (arbitrary example)
    mesh_noise.enable_instancing(2 * np.random.rand(1000, 3) - 1) # Add random instances of the primitive.  Each instance can also optionally have its own color and quaternion rotation.
    frame_hello.add_mesh(mesh_noise)
    
    # Create a frame on the hand canvas
    frame_hand = canvas_hand.create_frame(focus_point = hand_center)
    frame_hand.add_mesh(model_mesh_lo, transform = svt.Transforms.RotationAboutZAxis(math.cos(angle)))
    frame_hand.add_mesh(model_mesh_hi, transform = svt.Transforms.RotationAboutZAxis(math.cos(angle + math.pi)))
    
# Provide user with visibility controls (if not set then always visible)
canvas_hello.set_layer_settings({"Hello" : {"filled" : True }, "Analog Science" : {"filled" : True}, "Primitives" : {"filled" : False}, "Noise" : {"filled" : False, "opacity" : 0.5}})

# Save as self-contained html file
scene.save_as_html("Hello World.html", embed_library=True)

# Display here within Jupyter
scene